# Mario's Coursera Capstone Project

This notebook will be what I use for my capstone project.

In [1]:
# imports
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [3]:
from bs4 import BeautifulSoup
import requests

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# Getting the webpage, creating a Response object.
response = requests.get(url)

if response.status_code == 200:
    print('Success!')
elif response.status_code == 404:
    print('Not Found.')

data = response.content

Success!


In [4]:
soup = BeautifulSoup(data, 'lxml')

In [5]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":920980179,"wgRevisionId":920980179,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNames

In [6]:
table = soup.find('table')
print(table.prettify())

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

In [7]:
for row in table.find_all('tr'):
    try:
        print(row.find_all('td')[2].text)
    except:
        row.find('td') == None

Not assigned

Not assigned

Parkwoods

Victoria Village

Harbourfront

Regent Park

Lawrence Heights

Lawrence Manor

Not assigned

Not assigned

Islington Avenue

Rouge

Malvern

Not assigned

Don Mills North

Woodbine Gardens

Parkview Hill

Ryerson

Garden District

Glencairn

Not assigned

Not assigned

Cloverdale

Islington

Martin Grove

Princess Gardens

West Deane Park

Highland Creek

Rouge Hill

Port Union

Not assigned

Flemingdon Park

Don Mills South

Woodbine Heights

St. James Town

Humewood-Cedarvale

Not assigned

Not assigned

Bloordale Gardens

Eringate

Markland Wood

Old Burnhamthorpe

Guildwood

Morningside

West Hill

Not assigned

Not assigned

The Beaches

Berczy Park

Caledonia-Fairbanks

Not assigned

Not assigned

Not assigned

Woburn

Not assigned

Not assigned

Leaside

Central Bay Street

Christie

Not assigned

Not assigned

Not assigned

Cedarbrae

Hillcrest Village

Bathurst Manor

Downsview North

Wilson Heights

Thorncliffe Park

Adelaide

King

Rich

In [8]:
zipcodes = []

for row in table.find_all('tr'):
    zps = {}
    try:
        zps['postalcode'] = row.find_all('td')[0].text
        zps['borough'] = row.find_all('td')[1].text
        zps['neighborhood'] = row.find_all('td')[2].text.strip()
    except:
        row.find('td') == None
        
    zipcodes.append(zps)

toronto = pd.DataFrame(zipcodes)

In [9]:
# drop na's
toronto.dropna(inplace=True)

In [10]:
# remove rows with borough 'Not assigned'
toronto = toronto[toronto['borough'] != 'Not assigned']
toronto.head()

,borough,neighborhood,postalcode
3,North York,Parkwoods,M3A
4,North York,Victoria Village,M4A
5,Downtown Toronto,Harbourfront,M5A
6,Downtown Toronto,Regent Park,M5A
7,North York,Lawrence Heights,M6A


In [11]:
# find neighborhood 'Not assigned'
toronto[toronto['neighborhood'] == 'Not assigned']

,borough,neighborhood,postalcode
9,Queen's Park,Not assigned,M7A


In [12]:
# reassign neighborhood name to borough name
toronto['neighborhood'].loc[9] = toronto['borough'].loc[9]

In [13]:
# get neighborhoods into columns as lists
new = toronto.groupby(['postalcode', 'borough'])['neighborhood'].apply(list)

In [14]:
new = new.to_frame().reset_index()
new.head()

,postalcode,borough,neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


In [15]:
# remove brackets
new['neighborhood'] = new['neighborhood'].astype(str).str.strip('[]')
new.head()

,postalcode,borough,neighborhood
0,M1B,Scarborough,"'Rouge', 'Malvern'"
1,M1C,Scarborough,"'Highland Creek', 'Rouge Hill', 'Port Union'"
2,M1E,Scarborough,"'Guildwood', 'Morningside', 'West Hill'"
3,M1G,Scarborough,'Woburn'
4,M1H,Scarborough,'Cedarbrae'


In [16]:
# remove quotes
new['neighborhood'] = new['neighborhood'].str.replace("'", '')
new

,postalcode,borough,neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [17]:
# remove double quotes
new['neighborhood'] = new['neighborhood'].str.replace('"', '')
new

,postalcode,borough,neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [18]:
new.shape

(103, 3)

In [19]:
latlng = pd.read_csv('https://cocl.us/Geospatial_data')
latlng.rename(columns={'Postal Code':'postalcode'}, inplace=True)

In [20]:
toronto_final = pd.merge(new, latlng, on='postalcode')
toronto_final.head()

,postalcode,borough,neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [ ]:
# add creds

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['borough', 
                  'borough Latitude', 
                  'borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
toronto_venues = getNearbyVenues(names=toronto_final['borough'],
                                   latitudes=toronto_final['Latitude'],
                                   longitudes=toronto_final['Longitude']
                                  )

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [80]:
toronto_venues.head()

,borough,borough Latitude,borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Scarborough,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [81]:
toronto_venues.groupby(by='borough').count()

,borough Latitude,borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
borough,,,,,,
Central Toronto,115,115,115,115,115,115
Downtown Toronto,1299,1299,1299,1299,1299,1299
East Toronto,125,125,125,125,125,125
East York,74,74,74,74,74,74
Etobicoke,70,70,70,70,70,70
Mississauga,11,11,11,11,11,11
North York,240,240,240,240,240,240
Queen's Park,38,38,38,38,38,38
Scarborough,87,87,87,87,87,87


In [82]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


In [83]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['borough'] = toronto_venues['borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
toronto_onehot.shape

(2255, 276)

In [85]:
toronto_grouped = toronto_onehot.groupby('borough').mean().reset_index()
toronto_grouped

,borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.017391,0.00000,...,0.000000,0.008696,0.000000,0.000000,0.008696,0.000000,0.000000,0.000000,0.00000,0.008696
1,Downtown Toronto,0.000000,0.00077,0.000770,0.00077,0.00154,0.00077,0.00154,0.013087,0.00154,...,0.002309,0.011547,0.002309,0.000000,0.003849,0.000000,0.006159,0.000770,0.00077,0.003849
2,East Toronto,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.024000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.032000
3,East York,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.013514,0.000000,0.013514,0.000000,0.000000,0.00000,0.013514
4,Etobicoke,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.014286,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286,0.00000,0.000000
5,Mississauga,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.090909,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
6,North York,0.004167,0.00000,0.004167,0.00000,0.00000,0.00000,0.00000,0.008333,0.00000,...,0.000000,0.000000,0.004167,0.004167,0.008333,0.000000,0.000000,0.004167,0.01250,0.000000
7,Queen's Park,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,...,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.00000,0.026316
8,Scarborough,0.011494,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.011494,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.011494,0.000000,0.000000,0.000000,0.00000,0.000000
9,West Toronto,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,...,0.000000,0.005682,0.000000,0.000000,0.011364,0.000000,0.005682,0.000000,0.00000,0.005682


In [86]:
num_top_venues = 5

for hood in toronto_grouped['borough']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Park  0.06
3            Café  0.05
4    Dessert Shop  0.04


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.10
1                Café  0.05
2               Hotel  0.03
3  Italian Restaurant  0.03
4          Restaurant  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.06
1         Coffee Shop  0.06
2  Italian Restaurant  0.05
3                Café  0.04
4      Ice Cream Shop  0.04


----East York----
                 venue  freq
0          Coffee Shop  0.07
1         Burger Joint  0.05
2                 Park  0.04
3  Sporting Goods Shop  0.04
4                 Bank  0.04


----Etobicoke----
                  venue  freq
0           Pizza Place  0.11
1        Sandwich Place  0.07
2           Coffee Shop  0.07
3              Pharmacy  0.06
4  Fast Food Restaurant  0.04


----Mississauga----
                      venue  f

In [87]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [88]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['borough'] = toronto_grouped['borough']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Park,Sandwich Place,Café,Dessert Shop,Sushi Restaurant,Clothing Store,Gym,Pizza Place,Restaurant
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bakery,Bar,Park,Japanese Restaurant,Gastropub
2,East Toronto,Coffee Shop,Greek Restaurant,Italian Restaurant,Café,Ice Cream Shop,Yoga Studio,Brewery,Park,Bakery,Pizza Place
3,East York,Coffee Shop,Burger Joint,Pharmacy,Sporting Goods Shop,Bank,Park,Pizza Place,Liquor Store,Beer Store,Supermarket
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Park,Convenience Store,Beer Store


In [89]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [90]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('borough', 1)

# run k-means clustering
kmeans = KMeans(init='k-means++', n_clusters=kclusters, random_state=0, n_init=12).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 4, 0, 5, 1, 0, 3, 5, 4])

In [91]:
# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhoodtoronto_merged =toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('borough'), on='borough')
toronto_merged.head() # check the last columns!

,postalcode,borough,neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
3,M1G,Scarborough,Woburn,43.770992,-79.216917,5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank


In [92]:
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [93]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [94]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [95]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['postalcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [96]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
                     toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,postalcode,neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M2H,Hillcrest Village,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Park,Grocery Store,Restaurant,Bank,Café
18,M2J,"Fairview, Henry Farm, Oriole",0,Coffee Shop,Clothing Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Park,Grocery Store,Restaurant,Bank,Café
19,M2K,Bayview Village,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Park,Grocery Store,Restaurant,Bank,Café
20,M2L,"Silver Hills, York Mills",0,Coffee Shop,Clothing Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Park,Grocery Store,Restaurant,Bank,Café
21,M2M,"Newtonbrook, Willowdale",0,Coffee Shop,Clothing Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Park,Grocery Store,Restaurant,Bank,Café
22,M2N,Willowdale South,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Park,Grocery Store,Restaurant,Bank,Café
23,M2P,York Mills West,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Park,Grocery Store,Restaurant,Bank,Café
24,M2R,Willowdale West,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Park,Grocery Store,Restaurant,Bank,Café
25,M3A,Parkwoods,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Park,Grocery Store,Restaurant,Bank,Café
26,M3B,Don Mills North,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Park,Grocery Store,Restaurant,Bank,Café


In [97]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
                     toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,postalcode,neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,M7R,Canada Post Gateway Processing Centre,1,Coffee Shop,Hotel,Gym / Fitness Center,Middle Eastern Restaurant,Fried Chicken Joint,Mediterranean Restaurant,Sandwich Place,American Restaurant,Burrito Place,Dog Run


In [98]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
                     toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,postalcode,neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,M6C,Humewood-Cedarvale,2,Park,Convenience Store,Check Cashing Service,Tennis Court,Fast Food Restaurant,Field,Sandwich Place,Bus Line,Dog Run,Restaurant
74,M6E,Caledonia-Fairbanks,2,Park,Convenience Store,Check Cashing Service,Tennis Court,Fast Food Restaurant,Field,Sandwich Place,Bus Line,Dog Run,Restaurant
80,M6M,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",2,Park,Convenience Store,Check Cashing Service,Tennis Court,Fast Food Restaurant,Field,Sandwich Place,Bus Line,Dog Run,Restaurant
81,M6N,"The Junction North, Runnymede",2,Park,Convenience Store,Check Cashing Service,Tennis Court,Fast Food Restaurant,Field,Sandwich Place,Bus Line,Dog Run,Restaurant
98,M9N,Weston,2,Park,Convenience Store,Check Cashing Service,Tennis Court,Fast Food Restaurant,Field,Sandwich Place,Bus Line,Dog Run,Restaurant


In [99]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, 
                     toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,postalcode,neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,M7A,Queens Park,3,Coffee Shop,Park,Gym,Diner,Burger Joint,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burrito Place,Café


In [100]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, 
                     toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,postalcode,neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,The Beaches,4,Coffee Shop,Greek Restaurant,Italian Restaurant,Café,Ice Cream Shop,Yoga Studio,Brewery,Park,Bakery,Pizza Place
41,M4K,"The Danforth West, Riverdale",4,Coffee Shop,Greek Restaurant,Italian Restaurant,Café,Ice Cream Shop,Yoga Studio,Brewery,Park,Bakery,Pizza Place
42,M4L,"The Beaches West, India Bazaar",4,Coffee Shop,Greek Restaurant,Italian Restaurant,Café,Ice Cream Shop,Yoga Studio,Brewery,Park,Bakery,Pizza Place
43,M4M,Studio District,4,Coffee Shop,Greek Restaurant,Italian Restaurant,Café,Ice Cream Shop,Yoga Studio,Brewery,Park,Bakery,Pizza Place
50,M4W,Rosedale,4,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bakery,Bar,Park,Japanese Restaurant,Gastropub
51,M4X,"Cabbagetown, St. James Town",4,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bakery,Bar,Park,Japanese Restaurant,Gastropub
52,M4Y,Church and Wellesley,4,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bakery,Bar,Park,Japanese Restaurant,Gastropub
53,M5A,"Harbourfront, Regent Park",4,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bakery,Bar,Park,Japanese Restaurant,Gastropub
54,M5B,"Ryerson, Garden District",4,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bakery,Bar,Park,Japanese Restaurant,Gastropub
55,M5C,St. James Town,4,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bakery,Bar,Park,Japanese Restaurant,Gastropub


In [101]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, 
                     toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,postalcode,neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Rouge, Malvern",5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
1,M1C,"Highland Creek, Rouge Hill, Port Union",5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
2,M1E,"Guildwood, Morningside, West Hill",5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
3,M1G,Woburn,5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
4,M1H,Cedarbrae,5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
5,M1J,Scarborough Village,5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
7,M1L,"Clairlea, Golden Mile, Oakridge",5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
9,M1N,"Birch Cliff, Cliffside West",5,Breakfast Spot,Fast Food Restaurant,Chinese Restaurant,Bakery,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Thai Restaurant,Bank
